<a href="https://colab.research.google.com/github/ScottTeran/sandbox/blob/main/kaggle_tabular_jan_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRFRegressor, cv

In [2]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [3]:
# this is an aesthetic choice and just removes the many warnings that some functions and comands produce
# it helps significantly declutter the workbook
import warnings
warnings.filterwarnings('ignore')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26298 non-null  int64 
 1   date      26298 non-null  object
 2   country   26298 non-null  object
 3   store     26298 non-null  object
 4   product   26298 non-null  object
 5   num_sold  26298 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [5]:
train.head(5)

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


## feature engineering

In [6]:
# Credit to https://www.kaggle.com/ranjeetshrivastav/tps-jan-21-base-xgb
# and https://www.kaggle.com/bernhardklinger/tps-jan-2022/notebook

def feature_eng(df):
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    df['year'] = 'Y' + df['date'].dt.year.astype(str)
    df['quarter'] = 'Q' + df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    df.drop(columns=['date'],inplace=True)  

feature_eng(train)
feature_eng(test)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26298 entries, 0 to 26297
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_id     26298 non-null  int64 
 1   country    26298 non-null  object
 2   store      26298 non-null  object
 3   product    26298 non-null  object
 4   num_sold   26298 non-null  int64 
 5   week       26298 non-null  int64 
 6   year       26298 non-null  object
 7   quarter    26298 non-null  object
 8   day        26298 non-null  int64 
 9   dayofyear  26298 non-null  int64 
 10  weekend    26298 non-null  bool  
 11  weekday    26298 non-null  object
dtypes: bool(1), int64(5), object(6)
memory usage: 2.2+ MB


In [8]:
string_cols = [col for col in train.columns if train[col].dtype == 'object']

In [9]:
train = pd.get_dummies(train, columns=string_cols, drop_first=True)
test = pd.get_dummies(test, columns=string_cols, drop_first=True)

In [10]:
test.head(5)

,row_id,week,day,dayofyear,weekend,country_Norway,country_Sweden,store_KaggleRama,product_Kaggle Mug,product_Kaggle Sticker,quarter_Q2,quarter_Q3,quarter_Q4,weekday_WD1,weekday_WD2,weekday_WD3,weekday_WD4,weekday_WD5,weekday_WD6
0,26298,1,1,1,False,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,26299,1,1,1,False,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,26300,1,1,1,False,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,26301,1,1,1,False,0,0,1,1,0,0,0,0,1,0,0,0,0,0
4,26302,1,1,1,False,0,0,1,0,0,0,0,0,1,0,0,0,0,0


## train/test split

In [16]:
X = train.drop(columns='num_sold', axis=1)
y = train['num_sold']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)

## XGBoost model

In [65]:
# instantiate model
model_1 = XGBRFRegressor(n_estimators=75)
model_1.fit(X_train, y_train)

[22:44:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRFRegressor(n_estimators=75)

In [66]:
preds_1 = model_1.predict(X_valid)

In [67]:
mae_1 = mean_absolute_error(preds_1, y_valid)

print('Mean Absolute Error:', mae_1)

Mean Absolute Error: 84.01697487834741


In [68]:
# credit to https://www.statology.org/smape-python/

def smape(a, f): # 'a' for actual, and 'f' for forecast
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [69]:
smape(y_valid, preds_1)

19.61509762622562

## parameter tuning

In [21]:
model_2 = XGBRFRegressor()
model_2.fit(X_train, y_train)

[21:37:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRFRegressor()

In [33]:
cross_val_score(estimator=model_2, X=X_train, y=y_train).mean()

[22:14:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:14:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:14:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:14:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:14:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.7223892743093137

In [61]:
params = [{
  'n_estimators': [75], # tested 50,95...
}] 

xgb_gridsearch = GridSearchCV(
    estimator=model_2,
    param_grid=params,
    cv=5
)

In [62]:
xgb_gridsearch.fit(X_train, y_train)

[22:42:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:42:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, estimator=XGBRFRegressor(),
             param_grid=[{'n_estimators': [75]}])

In [63]:
xgb_gridsearch.best_params_

{'n_estimators': 75}

In [75]:
xgb_gridsearch.score(X_train, y_train)

0.7313949835152691

In [74]:
xgb_gridsearch.score(X_valid, y_valid)

0.7276904383936886

## test submission

In [76]:
submission = model_1.predict(test)

ValueError: ignored